In [1]:
%env QIBO_ENVIRONMENT = staging

env: QIBO_ENVIRONMENT=staging


In [1]:
from qiboconnection.connection import ConnectionConfiguration
from qiboconnection.api import API

[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:30:47]: Qibo Quantum Service API SERVER URL: https://qilimanjaro.ddns.net:8080
[Qibo 0.1.6|INFO|2022-01-28 16:30:47]: Using numpy backend on /CPU:0
[Qibo 0.1.6|WARNING|2022-01-28 16:30:47]: numpy backend uses `np.einsum` and supports CPU only. Consider installing the qibojit or qibotf backends for increased performance and to enable GPU acceleration.


## Connect to the Qibo Service API

In [8]:
qibo_api = API(configuration=ConnectionConfiguration({
      "user_id": 4,
      "username": "albert.solana",
      "api_key": "c3672747-2ea5-4451-8005-1f4f6588fa78",
}))


[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:01]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/authorisation-tokens
[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:01]: Connection successfully established.
[qibo-connection] 0.0.1|INFO|2022-01-28 16:50:01]: Storing personal qibo configuration...


## Load an already existing connection to the Qilimanjaro Service API

In [2]:
qibo_api = API()


[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:30:49]: Configuration loaded successfully.


## Checking if the connection is alive

In [9]:
qibo_api.ping()


[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:04]: Calling: https://qilimanjaro.ddns.net:8080/status


'OK'

## Listing all registered devices

In [10]:
devices = qibo_api.list_devices()


[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:08]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices


In [6]:
devices

<Devices[2]:
{
  "device_id": 1,
  "device_name": "radagast-simulator",
  "status": "online",
  "characteristics": {
    "type": "simulator",
    "cpu": "Intel Core i9-9900K @ 16x 5GHz",
    "gpu": "NVIDIA GeForce RTX 3090",
    "os": "Ubuntu 20.04 focal",
    "kernel": "x86_64 Linux 5.4.0-80-generic",
    "ram": "64185MiB"
  }
}
{
  "device_id": 7,
  "device_name": "galadriel-quantum",
  "status": "online",
  "characteristics": {
    "type": "quantum"
  },
  "calibration_details": {
    "t1": 13,
    "frequency": 988
  }
}

## Selecting a specific device

In [11]:
qibo_api.select_device_id(device_id=7)


[qibo-connection] 0.0.1|INFO|2022-01-28 16:50:18]: Device galadriel-quantum blocked.
[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:18]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/users/4
[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:19]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/messages?channel=1
[qibo-connection] 0.0.1|INFO|2022-01-28 16:50:19]: Device galadriel-quantum selected.


## Creating an experiment to be executed on a remote device

In [12]:
from qibo import gates
from qibo.core.circuit import Circuit

In [13]:
circuit = Circuit(1)
circuit.add(gates.X(0))
circuit.add(gates.M(0))


In [14]:
job_id = qibo_api.execute(circuit=circuit)

[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:32]: Sending qibo circuit for a remote execution...
[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:32]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/circuits
[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:33]: Job circuit queued successfully.


In [15]:
job_id

2261

In [16]:
result = qibo_api.get_result(job_id=job_id)


[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:50:38]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/jobs/2261
[qibo-connection] 0.0.1|INFO|2022-01-28 16:50:38]: Your job is completed.


In [18]:
result.state()

array([0.+0.j, 1.+0.j])

In [19]:
result.samples()

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [20]:
qibo_api.release_device(device_id=7)

[qibo-connection] 0.0.1|INFO|2022-01-28 16:52:25]: Device galadriel-quantum released.
[qibo-connection] 0.0.1|DEBUG|2022-01-28 16:52:25]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/messages?channel=1
